<img style="float:middle" src="https://images.moneycontrol.com/static-mcnews/2018/10/1280px-Indian_language_Wikipedias_word_cloud_based_on_number_of_articles-737x435.png?impolicy=website&width=770&height=431">

## 1. Introduction

I think this is going to be a real fun competition if I keep at it. Tamil is my favorite language and all the more reason to be excited for this one!

So let's begin the journey.....

This competition is about question and answers, where you need to predict answers to real questions on Wikipedia. Of course there is a lot of reasearch on this for English, but it's great that this will be expanded to other languages. And as per the problem statement this will help a population of 1.4 billion people since these languages are underepresented on the web. 

**Evaluation Metric**

The submissions will be evaluated based on the Jaccard metric as indicated in the competition details

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# read data
train = pd.read_csv('/kaggle/input/chaii-hindi-and-tamil-question-answering/train.csv')
test = pd.read_csv('/kaggle/input/chaii-hindi-and-tamil-question-answering/test.csv')

In [ ]:
# import libraries
from warnings import filterwarnings
filterwarnings('ignore')

pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

import plotly.figure_factory as ff
from plotnine import *
import plotnine as pn

import seaborn as sns

## 2. Data Understanding & Exploration

As always the first step is to get an understanding of the data. In this section we will explore the following -
* What is there in the data
* Overall size of the train and test dataset
* Nulls
* Word counts, text lengths
* Plot word clouds - just for fun. I can't read or understand Hindi :-) I will have no clue

### 2.1 Snapshot

From a snapshot of the training data the following can be inferred about the columns:

**id** - unique indetifier for each question-answer pair  
**context** - the text from which the answer needs to be derived  
**question** - the question for which the answer needs to be predicted  
**answer_text** - the answer to be predicted from the context. Will be available only for the train data  
**answer_start** - string position where the answer starts. Again only available for the train data.   
**language** - indicates which language the question-answer belongs to (there are only Hindi and Tamil currently)  

Based on an initial overview, id column will not be useful. The context, question and answer text all contain both English and Numbers. Something that must be kept in mind while text cleaning. Though the data should be very clean since the data is from Wikipedia.

The other thing is all the questions end with a question mark, which can be removed.

In [ ]:
display(train.head())

### 2.2 Train and Test Data Overview

In [ ]:
shape_df = pd.DataFrame({'Data':['Train','Test'],
                       'Shape':[train.shape[0], test.shape[0]]})
colors = ['#FFBF00','#40E0D0']
data = go.Bar(x =shape_df.Shape[::-1],y=shape_df.Data[::-1], orientation='h', text=shape_df.Shape[::-1], 
              textposition='auto', marker_color=colors)
layout = go.Layout(font=dict(family='Arial',size=14),
                  paper_bgcolor='white',
                  plot_bgcolor = '#FFFAFA',
                 showlegend=False,width=800, height=400,title='Train & Test Data Size')
fig = go.Figure(data=data, layout=layout)
fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black')
fig.show()

So only 1,114 data points on the train set. Transfer learning may be the correct approach? And just for fun they have given us only 5 test data points to predict on to evaluate. Surprised?

Based on the competition instructions they encourage to use additional data. That is something to consider.

### 2.3 Data Exploration

In this subsection we will deep dive into language distribution, word counts, text length and more!

In [ ]:
# language comparison
lang = train.groupby('language')['id'].count().reset_index().rename(columns={'language':'Language','id':'Count'})

colors = ['#FFBF00','#40E0D0']
data = go.Bar(x =lang.Language[::-1],y=lang.Count[::-1], text=lang.Count[::-1], 
              textposition='auto', marker_color=colors)

layout = go.Layout(font=dict(family='Arial',size=14),
                  paper_bgcolor='white',
                  plot_bgcolor = '#FFFAFA',
                 showlegend=False,width=800, height=400,title='Language Breakdown')

fig = go.Figure(data=data, layout=layout)
fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black')
fig.show()

67% of the data points belong to the Hindi. It may be worthwhile build a separate model for the two languages. And the test data maintains the distribution with 3 hindi entries and 2 tamil entries.

#### 2.3.1 Word Counts Comparison

In [ ]:
# add word and character count for context, question and answer text  
train['context_word_count'] = train['context'].apply(lambda x: len(x.split()))
train['question_word_count'] = train['question'].apply(lambda x: len(x.split()))
train['answer_word_count'] =  train['answer_text'].apply(lambda x: len(x.split()))

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(20, 6))

sns.kdeplot(train['context_word_count'],shade=True, color='#b03a2e', ax=axes[0])
sns.kdeplot(train[train.language == 'hindi']['context_word_count'],shade=True, color='#40E0D0', ax=axes[1],label='Hindi')
sns.kdeplot(train[train.language == 'tamil']['context_word_count'],shade=True, color='#FFBF00', ax=axes[1], label='Tamil')

axes[0].set(xlabel='', ylabel='')
axes[0].set_title('Context Word Count - Overall',fontdict= { 'fontsize': 12, 'fontweight':'bold'})

axes[1].set(xlabel='', ylabel='')
axes[1].set_title('Context Word Count - Hindi vs Tamil',fontdict= { 'fontsize': 12, 'fontweight':'bold'})
axes[1].legend(frameon=False, fontsize=12)

sns.despine()
plt.savefig('image1.png')

**Observations - Context Word Counts**
* Min number of words in the context: 24
* Max number of words in context: 10,259 available in the Hindi text
* The median word count for context is 1,075.
* The tamil contexts have a narrower distribution compared to the hindi contexts.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(20, 6))

sns.kdeplot(train['question_word_count'],shade=True, color='#b03a2e', ax=axes[0])
sns.kdeplot(train[train.language == 'hindi']['question_word_count'],shade=True, color='#40E0D0', ax=axes[1],label='Hindi')
sns.kdeplot(train[train.language == 'tamil']['question_word_count'],shade=True, color='#FFBF00', ax=axes[1], label='Tamil')

axes[0].set(xlabel='', ylabel='')
axes[0].set_title('Question Word Count - Overall',fontdict= { 'fontsize': 12, 'fontweight':'bold'})

axes[1].set(xlabel='', ylabel='')
axes[1].set_title('Question Word Count - Hindi vs Tamil',fontdict= { 'fontsize': 12, 'fontweight':'bold'})
axes[1].legend(frameon=False, fontsize=12)

sns.despine()

**Observations - Question Word Counts**
* The median word count in the questions is 7, pretty short questions. The max length for a question is 22 words.e
* The hindi language distribution is near normal whereas the tamil distribution for question word count is right skewed.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(20, 6))

sns.kdeplot(train['answer_word_count'],shade=True, color='#b03a2e', ax=axes[0])
sns.kdeplot(train[train.language == 'hindi']['answer_word_count'],shade=True, color='#40E0D0', ax=axes[1],label='Hindi')
sns.kdeplot(train[train.language == 'tamil']['answer_word_count'],shade=True, color='#FFBF00', ax=axes[1], label='Tamil')

axes[0].set(xlabel='', ylabel='')
axes[0].set_title('Answers Word Count - Overall',fontdict= { 'fontsize': 12, 'fontweight':'bold'})

axes[1].set(xlabel='', ylabel='')
axes[1].set_title('Answers Word Count - Hindi vs Tamil',fontdict= { 'fontsize': 12, 'fontweight':'bold'})
axes[1].legend(frameon=False, fontsize=12)

sns.despine()
plt.savefig('image3.png')

**Observations - Answer Word Counts**
* The median word count for the answers is 2. And 513 data points have an answer that is only 1 word Looks like the questions are specific to extract from the context. The max length for an an answer is 51. Again, since I do not know the language can't understand the context. Hopefully the NLP model will be better :-). We will also deep dive into single word answers to see how many of them are only numeric.
* The distributions for anwer word counts is comparable for both Hindi and Tamil

#### 2.3.2 Single Word Answers

In [ ]:
single_answers = train[train.answer_word_count == 1]
single_answers['is_numeric'] = single_answers['answer_text'].apply(lambda x: 1 if x.isnumeric() else 0)

single_answers['answer_type'] = np.where(single_answers.is_numeric == 1, 'Only Numeric','Other')

answer_type_count = single_answers.groupby('answer_type')['id'].count().reset_index().rename(columns={'answer_type':'Answer Type',
                                                                                                     'id':'Count'})
answer_type_count_lang = single_answers.groupby(['answer_type','language'])['id'].count().reset_index().rename(columns={'answer_type':'Answer Type',
                                                                                                     'id':'Count'})

answer_type_count_lang_other = answer_type_count_lang[answer_type_count_lang['Answer Type'] == 'Other']
answer_type_count_lang_numeric = answer_type_count_lang[answer_type_count_lang['Answer Type'] == 'Only Numeric']

fig = make_subplots(rows=1, cols=2, subplot_titles=(""),horizontal_spacing=0.05)

labels = ['Only Numeric','Other']
values = [133,380]

data = go.Pie(labels=labels, values=values,pull=[0.2,0],textinfo='label+value', marker=dict(colors=colors))
layout = go.Layout(font=dict(family='Arial',size=14),
                  paper_bgcolor='white',
                  plot_bgcolor = '#FFFAFA',
                 showlegend=False,title='Answer type for single word answers', height=500, width=500)
fig = go.Figure(data=data,layout=layout)
fig.show()

Approximately 25% of the answers overall are only nuemric, for example '206' or '1980'. The 'Other' type includes both only strings and strings plus numeric, for example 'C2H2'.

#### 2.3.3. Answer Start Position

This column indicates the start position of the string where the answer starts in the context. Not sure if this can be a feature. But let's explore. 

We will view this as a percentage, such as it is within the first 10% of the context, towards the end etc.

In [ ]:
# add start position pct
train['context_length'] = train['context'].apply(lambda x: len(x))
train['answer_pos'] = train['answer_start']/train['context_length']

train['answer_pos_cat'] = np.where(train['answer_pos']  < 0.01,'Start',
                                  np.where((train['answer_pos'] > 0.01) & (train['answer_pos'] <= 0.1),'Within 10%',
                                          np.where((train['answer_pos'] > 0.1) & (train['answer_pos'] <= 0.25),'10%-25%',
                                            np.where((train['answer_pos'] > 0.25) & (train['answer_pos'] <= 0.50),'25%-50%',
                                                np.where((train['answer_pos'] > 0.50) & (train['answer_pos'] <= 0.75),'50%-75%',
                                                    np.where((train['answer_pos'] > 0.75) & (train['answer_pos'] <= 0.90),'75%-90%',
                                                            'End'))))))

answer_start = train.answer_pos_cat.value_counts()
answer_start_df = pd.DataFrame({'Pos':answer_start.index,'Count':answer_start.values})

answer_pos_map = {'Start':1, 'Within 10%':2, '10%-25%':3, '25%-50%':4,'50%-75%':5,'75%-90%':6,'End':7}

answer_start_df['flag'] = answer_start_df['Pos'].map(answer_pos_map)
answer_start_df = answer_start_df.sort_values(by='flag')

answer_start_lang =  train.groupby(['language','answer_pos_cat'])['id'].count().reset_index().rename(columns={'id':'Count',
                                                                                                             'answer_pos_cat':'Pos'})
answer_start_hind = answer_start_lang[answer_start_lang.language == 'hindi']
answer_start_tamil = answer_start_lang[answer_start_lang.language == 'tamil']

answer_start_hind['flag'] = answer_start_hind['Pos'].map(answer_pos_map)
answer_start_tamil['flag'] = answer_start_tamil['Pos'].map(answer_pos_map)

answer_start_hind = answer_start_hind.sort_values(by='flag') 
answer_start_tamil = answer_start_tamil.sort_values(by='flag') 

fig = make_subplots(rows=1, cols=2,subplot_titles=('Overall', 'Hindi (Yellow) vs Tamil (Green)'))

fig.add_trace(go.Bar(x =answer_start_df.Count[::-1],y=answer_start_df.Pos[::-1], orientation='h', text=answer_start_df.Count[::-1], 
                          textposition='auto',
                          marker_color='#1f618d'),1,1)

fig.add_trace(go.Bar(x =answer_start_hind.Pos[::-1],y=answer_start_hind.Count[::-1], text=answer_start_hind.Count[::-1], 
                          textposition='auto',
                          marker_color='#FFBF00'),1,2)

fig.add_trace(go.Bar(x =answer_start_tamil.Pos[::-1],y=answer_start_tamil.Count[::-1], text=answer_start_tamil.Count[::-1], 
                          textposition='auto',
                          marker_color='#40E0D0'),1,2)


fig.update_layout(font=dict(family='Arial',size=14),
                  paper_bgcolor='white',
                  plot_bgcolor = '#FFFAFA',
                 showlegend=False,width=800, height=500,title='Answer Counts based on start position in context')

fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black')
fig.show()

plt.savefig('image2.png')

**Observations**
* The maximum number of answers are within 10% of the context string length. The distribution are similar for both Hindi and Tamil.



**Next Steps**
1. More EDA
2. Text Cleaning
3. Feature Engineering
4. And of course get into models

**Enjoy the journey and keep learning!**